In [ ]:
from __future__ import print_function

#coding=utf-8
import setproctitle
setproctitle.setproctitle('BeijingFlow@shaoerzhuo')

import os, tqdm, torch, sys
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from utils.datasets import DFGDataset, collate_fn
import utils.metrics as metrics
from model.correlation import get_CORR_numpy, get_CORR_numpy2, get_CORR_torch, get_CORR_torch2

class config:
    cuda_num = 6
    cityname = 'shanghai'
    io = 'outflow'
    time_slice = 48
    
    batch_size = 2**13
    interval = 50

checkin_cate = 14
poi_cate = 14

dataset_config = {
    'cityname' : config.cityname,
    'max_value' : {'beijing':{'inflow':1587.8500, 'outflow':1929.5500}, 'shanghai':{'inflow':8179.6667, 'outflow':9417.3333}}[config.cityname][config.io],
    'beta' : 1,
    'dataset_path' : os.path.join('/data2/shaoerzhuo/DeepFlowGen/Dataset', config.cityname, 'dataset'),
    'i/o' : config.io,
    'poi_cate' : poi_cate
}

print(dataset_config)

model_config = {
    'len_time_vec':64,
    'epoch_num' : {'beijing':{'inflow':5000, 'outflow':5000}, 'shanghai':{'inflow':5000, 'outflow':5000}}[config.cityname][config.io],
    'hidden_dims':{'beijing':256, 'shanghai':256}[config.cityname],
    'alpha':{'beijing':{'inflow':1e-3, 'outflow':3e-4}, 'shanghai':{'inflow':3e-4, 'outflow':1e-4}}[config.cityname][config.io],
    'lr':{'beijing':1e-3, 'shanghai':1e-3}[config.cityname],
}

print(model_config)

train_dataset = DFGDataset(dataset_config, 'train')
valid_dataset = DFGDataset(dataset_config, 'valid')
test_dataset = DFGDataset(dataset_config, 'test')

num_workers = 10
train_loader = DataLoader(train_dataset, num_workers=num_workers, batch_size=config.batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, num_workers=num_workers, batch_size=config.batch_size, shuffle=False, collate_fn=collate_fn)

In [ ]:
from sklearn.decomposition import NMF
from sklearn.linear_model import LinearRegression

nmf = NMF(14)
W = nmf.fit_transform(train_dataset.flow)
H = nmf.components_

In [ ]:
iacf = np.zeros([train_dataset.num_regions,48,14])
for iter in range(14):
    for iter2 in range(train_dataset.num_regions):
        iacf[iter2, :, iter] = W[iter2, iter] * H[iter]

pred_checkin = np.zeros([train_dataset.num_regions,48,14])
for iter in range(48):
    lr = LinearRegression()
    lr.fit(iacf[:, iter], train_dataset.checkin[:, iter])
    pred_checkin[:, iter] = lr.predict(iacf[:, iter])
print(get_CORR_numpy2(pred_checkin, train_dataset.checkin))

In [ ]:
class CrossLayer(nn.Module):
    def __init__(self, input_dim):
        super(CrossLayer, self).__init__()
        self.input_dim = input_dim
        self.weight = nn.Parameter(torch.zeros([self.input_dim]))
        self.bias = nn.Parameter(torch.zeros([self.input_dim]))
        torch.nn.init.normal_(self.weight.data, 0.0, 0.1)
        torch.nn.init.normal_(self.bias.data, 0.0, 0.1)
        
    def forward(self, x0, x):
        # [B, C]
        return x0 * torch.sum(x * self.weight, dim=1, keepdim=True) + self.bias + x
        #return torch.matmul(torch.bmm(torch.unsqueeze(x0, 2), torch.unsqueeze(xl, 1)), self.kernal) + self.bias + xl


class DeepFlowGen(nn.Module):
    def __init__(self):
        super(DeepFlowGen, self).__init__()
        self.hidden_dims = model_config['hidden_dims']
        self.time_dims = model_config['len_time_vec']
        self.cuda_num = config.cuda_num
        
        self.time_embedding_layer = nn.Embedding(48, self.time_dims)
        self.feat = nn.Sequential(nn.Linear(2 * poi_cate, self.hidden_dims), nn.Sigmoid())
        
        self.fcl0 = CrossLayer(self.hidden_dims + self.time_dims)
        self.fcl1 = CrossLayer(self.hidden_dims + self.time_dims)
        self.fcl2 = CrossLayer(self.hidden_dims + self.time_dims)
        self.fcl3 = CrossLayer(self.hidden_dims + self.time_dims)
        self.fcl4 = CrossLayer(self.hidden_dims + self.time_dims)
        
        self.fc0 = nn.Sequential(nn.Linear(self.hidden_dims + self.time_dims, self.hidden_dims), nn.Sigmoid())
        
        self.fc1 = nn.Sequential(nn.Linear(self.hidden_dims, self.hidden_dims), nn.Sigmoid())
        self.fc2 = nn.Sequential(nn.Linear(self.hidden_dims, self.hidden_dims), nn.Sigmoid())
        self.fc3 = nn.Sequential(nn.Linear(self.hidden_dims, self.hidden_dims), nn.Sigmoid())
        self.fc4 = nn.Sequential(nn.Linear(self.hidden_dims, self.hidden_dims), nn.Sigmoid())
        self.fc5 = nn.Sequential(nn.Linear(self.hidden_dims, self.hidden_dims), nn.Sigmoid())
        self.fc6 = nn.Sequential(nn.Linear(self.hidden_dims, self.hidden_dims), nn.Sigmoid())
            
        self.fc7 = nn.Sequential(nn.Linear(self.hidden_dims, checkin_cate))
        self.checkin_rate_embed_1 = nn.Embedding(48, checkin_cate)
        self.checkin_rate_embed_2 = nn.Embedding(48, checkin_cate)
        
        for m in self.modules():
            classname = m.__class__.__name__
            if classname.find('Linear') != -1:
                torch.nn.init.normal_(m.weight.data, 0.0, 0.1)
                torch.nn.init.normal_(m.bias.data, 0.0, 0.1)
            elif classname.find('Embedding') != -1:
                torch.nn.init.normal_(m.weight.data, 0.0, 0.1)
        
    def forward(self, batch):
        poi = batch['poi'].cuda(self.cuda_num)
        t = batch['t'].cuda(self.cuda_num)
        
        x0 = torch.cat([self.feat(poi), self.time_embedding_layer(t)[:, 0]], dim=1)
        x = x0
        #x = self.fcl0(x0, x)
        #x = self.fcl1(x0, x)
        #x = self.fcl2(x0, x)
        #x = self.fcl3(x0, x)
        #x = self.fcl4(x0, x)
        
        h = self.fc0(x)
        h = self.fc2(self.fc1(h) + h)
        h = self.fc4(self.fc3(h) + h)
        #h = self.fc6(self.fc5(h) + h)
        
        IACF = self.fc7(h)
        checkin_rate = torch.exp(self.checkin_rate_embed_1(t)[:, 0])
        checkin = IACF * checkin_rate
        
        return {'total_crowd_flow' : IACF.mean(-1, keepdim=True), 'IACF': IACF, 'checkin':checkin, 'checkin_rate' : checkin_rate}

In [ ]:
model = DeepFlowGen().cuda(config.cuda_num)
model.criterion = nn.MSELoss().cuda(config.cuda_num)
regularizetion_loss = 0
model.optimizer = optim.Adam(model.parameters(), lr=model_config['lr'], betas=(0.9, 0.999), eps=1e-8)
model.lr_scheduler = optim.lr_scheduler.ExponentialLR(model.optimizer, 0.5**(1/2000))

valid_list = [np.inf]
min_string = ''
for epoch in tqdm.tqdm(range(model_config['epoch_num']), ncols=70, ascii=True):
    model.train()
    for batch in train_loader:
        model.optimizer.zero_grad()
        return_dict = model(batch)
        p_total = return_dict['total_crowd_flow']
        t_total = batch['flow'].cuda(config.cuda_num)
        
        p_checkin = return_dict['checkin']
        t_checkin = batch['checkin'].cuda(config.cuda_num)
        
        checkin_rate = return_dict['checkin_rate']
        loss_1 = model.criterion(p_total, t_total)
        loss_2 = - model_config['alpha'] * get_CORR_torch2(p_checkin, t_checkin)
        #loss_3 = - torch.sum(checkin_rate*torch.log(checkin_rate), dim=-1).mean()
        loss = loss_1 + loss_2# + loss_3
        loss.backward(retain_graph=True)
        model.optimizer.step()
    model.lr_scheduler.step()
    if epoch % config.interval == config.interval-1:
        model.eval()
        print('Loss 1 = \t{0:.4f}\tLoss 2 = \t{1:.4f}\t'.format(loss_1, loss_2))
        pred_total = []
        target_total = []
        pred_checkin = []
        target_checkin = []
        for batch in train_loader:
            pred_total.append(model(batch)['total_crowd_flow'].detach().cpu().numpy())
            target_total.append(batch['flow'].numpy())
            pred_checkin.append(model(batch)['checkin'].detach().cpu().numpy())
            target_checkin.append(batch['checkin'].numpy())
        pred_total = np.concatenate(pred_total, axis=0)
        target_total = np.concatenate(target_total, axis=0)
        pred_checkin = np.concatenate(pred_checkin, axis=0)
        target_checkin = np.concatenate(target_checkin, axis=0)
        
        MAE = metrics.get_MAE(pred_total, target_total) * dataset_config['max_value']
        RMSE = metrics.get_RMSE(pred_total, target_total) * dataset_config['max_value']
        NRMSE = metrics.get_NRMSE(pred_total, target_total)
        CORR = get_CORR_numpy2(pred_checkin, target_checkin)
        print('Epoch={0}\tMAE=\t{1:.6f}\tRMSE=\t{2:.6f}\tNRMSE=\t{3:.6f}\tCORR=\t{4:.6f}'.format(epoch, MAE, RMSE, NRMSE, CORR))
        
        
        pred_total = []
        target_total = []
        pred_checkin = []
        target_checkin = []
        for batch in valid_loader:
            pred_total.append(model(batch)['total_crowd_flow'].detach().cpu().numpy())
            target_total.append(batch['flow'].numpy())
            pred_checkin.append(model(batch)['checkin'].detach().cpu().numpy())
            target_checkin.append(batch['checkin'].numpy())
        pred_total = np.concatenate(pred_total, axis=0)
        target_total = np.concatenate(target_total, axis=0)
        pred_checkin = np.concatenate(pred_checkin, axis=0)
        target_checkin = np.concatenate(target_checkin, axis=0)
        
        MAE = metrics.get_MAE(pred_total, target_total) * dataset_config['max_value']
        RMSE = metrics.get_RMSE(pred_total, target_total) * dataset_config['max_value']
        NRMSE = metrics.get_NRMSE(pred_total, target_total)
        CORR = get_CORR_numpy2(pred_checkin, target_checkin)
        print('Epoch={0}\tMAE=\t{1:.6f}\tRMSE=\t{2:.6f}\tNRMSE=\t{3:.6f}\tCORR=\t{4:.6f}'.format(epoch, MAE, RMSE, NRMSE, CORR))
        
torch.save(model, 'checkpoints/DFG-DCN-{0}-{1}.pkl'.format(config.cityname, config.io))

In [ ]:
pred_IACF = []
target_checkin = []
for batch in valid_loader:
    pred_IACF.append(model(batch)['IACF'].detach().cpu().numpy())
    target_checkin.append(batch['checkin'].numpy())
pred_IACF = np.concatenate(pred_IACF, axis=0)
target_checkin = np.concatenate(target_checkin, axis=0)

CORR = get_CORR_numpy2(pred_IACF, target_checkin)

In [ ]:
CORR